In [324]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import logging
import dspy
import os
from dotenv import load_dotenv

In [325]:
# Show all rows
pd.set_option('display.max_rows', None)

# Show all columns
pd.set_option('display.max_columns', None)

# Prevent column width truncation
pd.set_option('display.max_colwidth', None)  # or use pd.set_option('display.max_colwidth', -1) in older versions

# Prevent overall frame width truncation
pd.set_option('display.width', None)

In [326]:
logger = logging.getLogger(__name__)
logging.basicConfig(filename='example.log', encoding='utf-8', level=logging.ERROR)

In [327]:
df = pd.read_csv('ch.csv')
links = df[df['Content'].str.contains('http',na=False)]

In [328]:
filtered = links[~links['Content'].str.contains('youtu|.pdf', na=False)]
filtered.reset_index(inplace=True)

In [329]:
def scrape_data(index,url):
    try:
        data = requests.get(url).text
        soup = BeautifulSoup(data)
    except Exception as e:
        logger.error(f"Could not parse URL index {index} : {url}\nError_message: {e} \n")
        return ''
    return soup

In [330]:
data = []
for index,i in enumerate(filtered.Content) :
    data.append(scrape_data(index,i))

C:\Users\avira\AppData\Local\Temp\ipykernel_14472\1171803274.py:4: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(data)


In [331]:
text = []
for i in data:
    try:
        text.append(i.text.replace('\n',''))
    except Exception as e:
        logger.error(f"Couldnt parse data from the website:\n Error_message:{e} ")

In [332]:
load_dotenv()


True

In [333]:
lm = dspy.LM("groq/qwen/qwen3-32b", api_key= os.getenv("GROQ_API_KEY"))
dspy.configure(lm=lm)

In [334]:
from typing import Literal


class ResourceDetails(dspy.Signature):
    """Extract detailed information from a technology-related content resource (e.g., blog, video, tutorial, research paper)."""
    
    resource_text: str = dspy.InputField()

    title: str = dspy.OutputField(desc="Title of the content resource")
    author_or_creator: str = dspy.OutputField(desc="Name of the author, creator, or publisher")
    content_type: Literal['Video', 'Blog', 'Research Paper', 'Tutorial','Roadmap', 'Other'] = dspy.OutputField(desc="Type of the content")
    
    tech_field: str = dspy.OutputField(desc="Main field of technology: e.g., AI, Data Science, Web Dev, Cloud, etc.")
    subfield: str = dspy.OutputField(desc="Subfield of the main tech domain: e.g., NLP, LLM, Vision, Backend, Frontend, MLOps, etc.")
    
    programming_languages: str = dspy.OutputField(desc="Programming languages mentioned or used in the content (e.g., Python, JavaScript, Rust, etc.)")
    technical_tools_and_libraries: str = dspy.OutputField(desc="Tools, frameworks, and libraries referenced (e.g., TensorFlow, PyTorch, React, Docker)")
    
    summary: str = dspy.OutputField(desc="3-line concise summary of the resource’s purpose and content")
    
    difficulty_level: Literal['Beginner', 'Intermediate', 'Advanced'] = dspy.OutputField(desc="Target skill level required for understanding the content")
    publication_date: str = dspy.OutputField(desc="When the resource was published or created")
    estimated_time_to_complete: str = dspy.OutputField(desc="Estimated time to watch/read/complete the resource")
    
    tags: str = dspy.OutputField(desc="Relevant tags or keywords for the content")
    resource_link: str = dspy.OutputField(desc="URL or location to access the resource")

    notable_insights: str = dspy.OutputField(desc="Unique ideas, breakthroughs, or standout techniques introduced in the resource")


In [335]:
resource_parser = dspy.ChainOfThought(ResourceDetails)

In [336]:
from transformers import pipeline
import numpy as np
classifier = pipeline("zero-shot-classification", "r-f/ModernBERT-large-zeroshot-v1")


Device set to use cpu


In [337]:
def classify_accessibility(text):
    candidate_labels = ["accessible","not_accessible"]
    output = classifier(text, candidate_labels, multi_label=False)
    return output['labels'][np.argmax(output['scores'])]


In [338]:
import pandas as pd
import time
first_iter = True
for t in text:
    if classify_accessibility(t) == 'accessible':
        try:
            data = resource_parser(resource_text=t)
        except Exception as e:
            logger.error(f"failed to parse data {e}")
            continue 
        time.sleep(15)
        row_df = pd.DataFrame([data.toDict()])  
        if first_iter:
            df = row_df
            first_iter = False
        else:
            df = pd.concat([df, row_df], ignore_index=True)  

df.to_csv('resources.csv', index=False)


2025/08/06 19:36:07 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/06 19:42:13 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/06 19:43:46 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/06 19:47:32 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/06 19:48:29 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/06 19:49:24 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/06 19:51:37 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/06 19:53:57 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/06 19:57

In [339]:
dt = pd.read_csv('resources.csv')

In [340]:
dt

,reasoning,title,author_or_creator,content_type,tech_field,subfield,programming_languages,technical_tools_and_libraries,summary,difficulty_level,publication_date,estimated_time_to_complete,tags,resource_link,notable_insights
0,"The resource is a Java and Object-Oriented Programming (OOP) cheat sheet created by son9912. It covers core Java syntax, OOP concepts, data types, operators, control structures, string handling, input methods, access modifiers, abstract classes, interfaces, encapsulation, inheritance, and collections. The content is structured for quick reference, with practical examples and explanations tailored for learners. The publication date is explicitly stated as September 25th, 2017, and the resource is categorized as a cheat sheet, which falls under the ""Other"" content type. The technical depth and examples suggest an intermediate difficulty level.",Java + OOP Concept Cheat Sheet,son9912,Other,Java,Object-Oriented Programming (OOP),Java,"java.util.Scanner, java.lang.Math, java.util.Collection (HashSet, LinkedList, HashMap)","This cheat sheet provides a concise reference for Java programming fundamentals and OOP concepts. It includes syntax examples for variables, loops, conditionals, string manipulation, input handling, access modifiers, abstract classes, interfaces, and collection frameworks. Designed for quick lookup, it emphasizes practical code snippets and memory optimization techniques like the String Pool. Targeted at learners transitioning from basics to intermediate Java development.",Intermediate,"25th September, 2017",Not specified (likely 15-30 minutes for review),"Java, OOP, programming, cheat sheet, object-oriented, collections, access modifiers, inheritance, encapsulation",https://cheatography.com/son9912/cheat-sheets/java-oop-concept/,"- Practical examples for Java syntax and OOP principles.\n- Explanation of memory optimization via String Pool.\n- Clear differentiation between abstract classes and interfaces.\n- Coverage of collection frameworks (HashSet, LinkedList, HashMap) with usage examples.\n- Emphasis on naming conventions and access modifiers for code readability."
1,"The resource is a structured learning roadmap for mastering Data Structures and Algorithms (DSA) provided by GeeksforGeeks. It outlines a 5-step process, including language prerequisites, core concepts, libraries, and problem-solving strategies. The content is organized into sequential phases, targeting beginners and emphasizing practical implementation. Key details like programming languages, tools, and subfields are explicitly mentioned in the text.",Complete Roadmap To Learn DSA,GeeksforGeeks,Roadmap,Data Structures and Algorithms,"Data Structures, Algorithms","Python, Java, C++, JavaScript","Standard Template Library (STL) for C++, Java's java.util package, Python's collections and heapq modules","This roadmap provides a structured 5-step guide to learn DSA, starting with programming language fundamentals, progressing to core DSA concepts, and culminating in advanced problem-solving. It emphasizes practical implementation, complexity analysis, and leveraging built-in libraries for efficiency. Designed for beginners, it prepares learners for technical interviews and real-world applications.",Beginner,"27 Jul, 2025",Not explicitly specified (self-paced),"DSA, Data Structures, Algorithms, Learning Roadmap, Technical Interview Prep, Programming Fundamentals",https://www.geeksforgeeks.org/complete-roadmap-to-learn-dsa/,"- Structured 5-step approach: language basics → logic building → DSA fundamentals → libraries → advanced problem-solving.\n- Emphasis on complexity analysis (Big O, Omega, Theta notations).\n- Language-specific prerequisites for C, C++, Java, Python, and JavaScript.\n- Integration of practical problem-solving via platforms like GeeksforGeeks and the SDE Sheet for interview preparation."
2,"The resource is a comprehensive JavaScript tutorial titled ""The Modern JavaScript Tutorial,"" covering both foundati